In [1]:
!pip3 install tensorflow==2.0.0-rc1


     |████████████████████████████████| 86.3MB 122kB/s 
     |████████████████████████████████| 4.3MB 42.5MB/s 
     |████████████████████████████████| 501kB 51.0MB/s 
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [2]:
import tensorflow as tf
print ("check tensorflow version: ", tf.__version__)

check tensorflow version:  2.0.0-rc1


This notebook is a basic tutorial for setting features and labels for time series analysis. Feature is effectively a number of values in the series,
with label being the next value. We'll call that number of values that will treat as our feature, the window size, where we're taking a window of the data and training an ML model to predict the next value. So for example, if we take our time series data, say, 30 days at a time, we'll use 30 values as the feature and the next value is the label. Then over time, we'll train a neural network to match the 30 features to the single label. 

In [3]:
dataset = tf.data.Dataset.range(14)
for val in dataset:
   print(val.numpy()) # because tensroflow 2 supports eager execution, so no need to start session

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [5]:
dataset = tf.data.Dataset.range(14)
dataset = dataset.window(5, shift=2) # now select a window of 5, where our idea is that last value will be label, first 4 being the features.
for window_dataset in dataset:
  for val in window_dataset:
    print(val.numpy(), end=" ")
  print()

0 1 2 3 4 
2 3 4 5 6 
4 5 6 7 8 
6 7 8 9 10 
8 9 10 11 12 
10 11 12 13 
12 13 


In [6]:
dataset = tf.data.Dataset.range(14)
dataset = dataset.window(5, shift=1, drop_remainder=True) # choose only the entries that contain 5 elements
for window_dataset in dataset:
  for val in window_dataset:
    print(val.numpy(), end=" ")
  print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 
6 7 8 9 10 
7 8 9 10 11 
8 9 10 11 12 
9 10 11 12 13 


In [7]:
dataset = tf.data.Dataset.range(14)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
for window in dataset:
  print(window.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]
[ 6  7  8  9 10]
[ 7  8  9 10 11]
[ 8  9 10 11 12]
[ 9 10 11 12 13]


In [8]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))# last but all the elements are features 
for x,y in dataset:
  print(x.numpy(), y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]


In [10]:
dataset = tf.data.Dataset.range(14)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=14) # shuffle the data-set, buffer size = dataset size
for x,y in dataset:
  print(x.numpy(), y.numpy())

[2 3 4 5] [6]
[ 8  9 10 11] [12]
[0 1 2 3] [4]
[6 7 8 9] [10]
[ 9 10 11 12] [13]
[1 2 3 4] [5]
[5 6 7 8] [9]
[ 7  8  9 10] [11]
[3 4 5 6] [7]
[4 5 6 7] [8]


In [12]:
dataset = tf.data.Dataset.range(14)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=14)
dataset = dataset.batch(3).prefetch(1) # a batch of 3 inputs 
for x,y in dataset:
  print("x = ", x.numpy())
  print("y = ", y.numpy())

x =  [[3 4 5 6]
 [6 7 8 9]
 [4 5 6 7]]
y =  [[ 7]
 [10]
 [ 8]]
x =  [[ 2  3  4  5]
 [ 9 10 11 12]
 [ 8  9 10 11]]
y =  [[ 6]
 [13]
 [12]]
x =  [[ 7  8  9 10]
 [ 1  2  3  4]
 [ 5  6  7  8]]
y =  [[11]
 [ 5]
 [ 9]]
x =  [[0 1 2 3]]
y =  [[4]]
